In [1]:
import numpy as np
import pandas as pd
import keras

from fi2010 import fetch_fi2010
import LobAttention, LobFeatures, LobPosition, LobTransformer
pd.options.mode.copy_on_write = True

# data

In [2]:
df = fetch_fi2010()

In [10]:
np.array(df[(df['DAY']==1)&(df['STOCK']==1)].loc[0])

array([   1,    1, 2615, 2618, 2619, 2620, 2621, 2623, 2625, 2626, 2629,
       2633, 2606, 2604, 2602, 2600, 2599, 2595, 2593, 2591, 2588, 2579,
        353,  200,  164,  532,  151,  837,  150,  787,  146,  311,  326,
        682,  786,  893,  159,  100,  143,  134,  123,  128,    0,    0,
          0,    0,    0], dtype=int64)

In [13]:
price = [ i for i in df.columns if 'PRICE' in i]
volume = [ i for i in df.columns if 'VOLUME' in i]

In [21]:
need_cols = list(sum(zip(price,volume),()))

In [24]:
np.array(df[need_cols].loc[0])

array([2615,  353, 2618,  200, 2619,  164, 2620,  532, 2621,  151, 2623,
        837, 2625,  150, 2626,  787, 2629,  146, 2633,  311, 2606,  326,
       2604,  682, 2602,  786, 2600,  893, 2599,  159, 2595,  100, 2593,
        143, 2591,  134, 2588,  123, 2579,  128], dtype=int64)

In [3]:
def make_simple_data(
    df: pd.DataFrame,
    group_by=['STOCK', 'DAY'],
    last_n=100,
):

    columns_price = [i for i in df.columns if 'PRICE' in i]
    columns_volume = [i for i in df.columns if 'VOLUME' in i]
    columns =\
        columns_price[:10] +\
        columns_volume[:10] +\
        columns_price[10:] +\
        columns_volume[10:]

    result = np.array(
        [i[columns].to_numpy()[-last_n:] for _, i in df.groupby(by=group_by)])

    return result

In [4]:
# def make_sequence_tensor(df_stock:pd.DataFrame):
#     columns_volume = [i for i in df_stock.columns if 'VOLUME' in i]
#     columns_volume = columns_volume[-11::-1] + columns_volume[10:]
#     columns_price = [i for i in df_stock.columns if 'PRICE' in i]
#     columns_price = columns_price[-11::-1] + columns_price[10:]
#     df_price = (
#         df_stock[columns_price]
#         .stack()
#         .to_frame(name='price')
#         .reset_index(level=[1])
#         .drop(columns='level_1')) # yapf: disable

#     df_volume = (
#         df_stock[columns_volume]
#         .stack()
#         .to_frame(name='volume')
#         .reset_index(level=[1])
#         .drop(columns='level_1'))# yapf: disable
    
#     book = df_price
#     book['volume'] = df_volume
#     data = book.to_numpy().reshape(-1, 20, 2)
#     return data

In [5]:
data = make_simple_data(df)

In [6]:
data.shape

(50, 100, 40)

# Model

In [11]:
inp = keras.Input(shape=(100,40),)

In [12]:
lob_dilated = LobFeatures.lob_dilated(inp)
norm = LobTransformer.LayerNormalization()(lob_dilated)
encoder = LobPosition.positional_encoding(norm)
trans1 =LobTransformer.TransformerBlock(name='1',num_heads=3)

In [13]:
trans1(encoder)

ValueError: Inputs have incompatible shapes. Received shapes (100, 15) and (100, 5)

In [17]:
a= encoder

In [18]:
a

<KerasTensor: shape=(5, 100, 15) dtype=float32 (created by layer 'tf.concat_1')>